# Plot groups' analysis
## Set parameters

In [ ]:
# ####################################### GENERAL OPTIONS #######################################
#EXPERIMENT_ROOT = "/run/user/1000/gvfs/smb-share:server=ich.techosp.it,share=ricerca/Lab Matteoli/Silva/collaborations/Mathias/soumnya/data/experiment/"
EXPERIMENT_ROOT = "../data/experiments/soumnya/experiment/"
#PROOF_ROOT = "/run/user/1000/gvfs/smb-share:server=ich.techosp.it,share=ricerca/Lab Matteoli/Silva/collaborations/Mathias/soumnya/data/proof/"
PROOF_ROOT = "../data/experiments/soumnya/proof/"
#PLOTS_OUTPUT_PATH = "/run/user/1000/gvfs/smb-share:server=ich.techosp.it,share=ricerca/Lab Matteoli/Silva/collaborations/Mathias/soumnya/results/"
PLOTS_OUTPUT_PATH = "./plots/soumnya/"

mode = "sum"
MARKER = "CFos"
PLOT_HEIGHT = 900
PLOT_WIDTH = 900
PLOT_MAX_X = 4000
PLOT_MAX_Y = 4000
SHOW_PLOT = True
SAVE_PLOT = False
SAVED_PLOT_EXTENSION = ".html"                      # '.html' for interactive plot
                                                    # '.svg' for vectorized image
                                                    # '.png'/'.jpg'/... for rasterized image

## Script's code
run all cell below

In [ ]:
import os
import sys

project_path = os.path.dirname(os.path.abspath(os.getcwd()))
sys.path.append(project_path)
import BraiAn

experiment_input_path = os.path.join(EXPERIMENT_ROOT, "BraiAn_output")
proof_input_path = os.path.join(PROOF_ROOT, "BraiAn_output")

In [ ]:
animals_sums = []

for file in os.listdir(proof_input_path):
    if os.path.isfile(os.path.join(proof_input_path, file)) and file.endswith(f"_{mode}.csv"):
        animal = file[:-len(f"_{mode}.csv")]
        experiment_animal = BraiAn.AnimalBrain.from_csv(animal, experiment_input_path, mode=mode)
        proof_animal = BraiAn.AnimalBrain.from_csv(animal, proof_input_path, mode=mode)
        animals_sums.append((BraiAn.AnimalBrain.merge_hemispheres(experiment_animal), BraiAn.AnimalBrain.merge_hemispheres(proof_animal)))

In [ ]:
animals = []
regions = []
experiment_density = []
proof_density = []
for experiment_animal, proof_animal in animals_sums:
    proof_animal_densities = proof_animal.data[MARKER] / proof_animal.data["area"]
    experiment_animal_densities = experiment_animal.data[MARKER] / experiment_animal.data["area"] 
    for brain_region in proof_animal_densities.index:
        if brain_region in experiment_animal_densities.index:
            animals.append(experiment_animal.name)
            regions.append(brain_region)
            experiment_density.append(experiment_animal_densities[brain_region])
            proof_density.append(proof_animal_densities[brain_region])


In [ ]:
import plotly.graph_objects as go

hover_text = [f"<b>animal</b>: {animal}<br><b>region</b>: {region}<br><b>experiment</b>: {experiment}<br><b>proof</b>: {proof}" for animal, region, experiment, proof in zip(animals, regions, experiment_density, proof_density)]

fig = go.Figure([
    go.Scatter(
        x=experiment_density,
        y=proof_density,
        text=hover_text,
        hoverinfo='text',
        mode="markers"
    )]
)

fig.update_layout(
    xaxis=dict(
        title="experiment",
        range=[0, PLOT_MAX_X]
    ),
    yaxis=dict(
        title="proof",
        range=[0, PLOT_MAX_Y]
    ),
    height=PLOT_HEIGHT,
    width=PLOT_WIDTH
)

if SAVE_PLOT:
    plot_filename = f"correlation_experiment_vs_proof{SAVED_PLOT_EXTENSION}".lower()
    plot_filepath = os.path.join(PLOTS_OUTPUT_PATH, plot_filename)
    match SAVED_PLOT_EXTENSION.lower():
        case ".html":
            fig.write_html(plot_filepath)
        case _:
            fig.write_image(plot_filepath)

if SHOW_PLOT:
    fig.show()

In [ ]:
from scipy.stats import pearsonr
r,p = pearsonr(experiment_density, proof_density)
r,p

In [ ]:
from collections import namedtuple
GroupDirectory = namedtuple("GroupDirectory", "name dirs")

groups = [
    GroupDirectory(
        name="Control (Females)",
        dirs=["42CF", "44CF", "50CF", "52CF", "58CF", "60CF", "74CF", "76CF"]
    ),
    GroupDirectory(
        name="Stress (Females)",
        dirs=["46SF", "48SF", "54SF", "56SF", "62SF", "64SF", "78SF", "80SF"]
    ),
    GroupDirectory(
        name="Control (Males)",
        dirs=["41CM", "43CM", "49CM", "51CM", "81CM", "83CM", "89CM", "91CM"]
    ),
    GroupDirectory(
        name="Stress (Males)",
        dirs=["45SM", "47SM", "53SM", "55SM", "85SM", "87SM", "93SM", "95SM"]
    ),
]

In [ ]:
animal_groups = []

for group in groups:
    animals_sums = []
    for file in os.listdir(proof_input_path):
        if os.path.isfile(os.path.join(proof_input_path, file)) and file.endswith(f"_{mode}.csv"):
            animal = file[:-len(f"_{mode}.csv")]
            if animal not in group.dirs:
                continue
            experiment_animal = BraiAn.AnimalBrain.from_csv(animal, experiment_input_path, mode=mode)
            proof_animal = BraiAn.AnimalBrain.from_csv(animal, proof_input_path, mode=mode)
            animals_sums.append((BraiAn.AnimalBrain.merge_hemispheres(experiment_animal), BraiAn.AnimalBrain.merge_hemispheres(proof_animal)))
    animal_groups.append((group.name, animals_sums))

In [ ]:
correlations = []
for group_name, animals_sums in animal_groups:
    animals = []
    regions = []
    experiment_density = []
    proof_density = []
    for experiment_animal, proof_animal in animals_sums:
        proof_animal_densities = proof_animal.data[MARKER] / proof_animal.data["area"]
        experiment_animal_densities = experiment_animal.data[MARKER] / experiment_animal.data["area"] 
        for brain_region in proof_animal_densities.index:
            if brain_region in experiment_animal_densities.index and brain_region in ["HY", "TH"]: #"root":
                animals.append(experiment_animal.name)
                regions.append(brain_region)
                experiment_density.append(experiment_animal_densities[brain_region])
                proof_density.append(proof_animal_densities[brain_region])
    
    hover_text = [f"<b>animal</b>: {animal}<br><b>region</b>: {region}<br><b>experiment</b>: {experiment}<br><b>proof</b>: {proof}" for animal, region, experiment, proof in zip(animals, regions, experiment_density, proof_density)]

    fig = go.Figure([
        go.Scatter(
            x=experiment_density,
            y=proof_density,
            text=hover_text,
            hoverinfo='text',
            mode="markers"
        )]
    )

    fig.update_layout(
        title=group_name,
        xaxis=dict(
            title="experiment",
        ),
        yaxis=dict(
            title="proof",
        ),
        height=PLOT_HEIGHT,
        width=PLOT_WIDTH
    )

    if SAVE_PLOT:
        plot_filename = f"correlation_experiment_vs_proof{SAVED_PLOT_EXTENSION}".lower()
        plot_filepath = os.path.join(PLOTS_OUTPUT_PATH, plot_filename)
        match SAVED_PLOT_EXTENSION.lower():
            case ".html":
                fig.write_html(plot_filepath)
            case _:
                fig.write_image(plot_filepath)

    if SHOW_PLOT:
        fig.show()

    r,p = pearsonr(experiment_density, proof_density)
    correlations.append((group_name, r, p))


In [ ]:
for group_name, r, p in correlations:
    print(f"{group_name} --- r: {r}, p: {p}")